In [1]:
!pip install fasttext
!pip install xgboost

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [2]:
import pandas as pd
import nltk
from nltk import FreqDist
from nltk.tokenize import word_tokenize, sent_tokenize


import fasttext
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

import csv

import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

# Make sure you've downloaded the appropriate resources
nltk.download('punkt')

np.random.seed(42)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
lang = "en"

df = pd.read_csv("../data/subtask_1/"+lang+"/train.tsv", sep='\t')


mapping = {
    "generated":0,
    "human":1
}

df["label"] = df['label'].map(mapping)
df=df.drop(df.columns[0], axis=1)

In [4]:
def calculate_features(text):
    word_tokens = word_tokenize(text)
    sent_tokens = sent_tokenize(text)
    
    avg_word_length = sum(len(word) for word in word_tokens) / len(word_tokens)
    
    avg_sent_length = sum(len(sent) for sent in sent_tokens) / len(sent_tokens)
    
    vocab_richness = len(set(word_tokens)) / len(word_tokens)
    
    freq_dist = FreqDist(word_tokens)
    repetition_rate = len([freq for word, freq in freq_dist.items() if freq > 1]) / len(word_tokens)

    return avg_word_length, avg_sent_length, vocab_richness, repetition_rate


df['avg_word_length'], df['avg_sent_length'], df['vocab_richness'], df['repetition_rate'] = zip(*df['text'].map(calculate_features))


In [5]:
df.head()

,text,label,avg_word_length,avg_sent_length,vocab_richness,repetition_rate
0,you need to stop the engine and wait until it ...,0,3.120000,49.500000,0.960000,0.040000
1,The Commission shall publish the report; an in...,0,4.919192,62.555556,0.686869,0.090909
2,"I have not been tweeting a lot lately, but I d...",0,3.160000,49.500000,0.840000,0.120000
3,I pass my exam and really thankgod for that bu...,1,3.550000,90.000000,0.900000,0.100000
4,The template will have 3 parts: a mustache sha...,1,3.985294,65.000000,0.632353,0.191176


In [6]:
RANDOM_SEED = 42


train_df, test_df = train_test_split(df, test_size=0.1, random_state=42)
train_df, valid_df = train_test_split(train_df, test_size=0.1, random_state=42)


train_df['text'].fillna('', inplace=True)
train_df['label'].fillna('0', inplace=True)  

train_df[['label', 'text']].to_csv('train.txt', index=False, sep=' ', header=False, quoting=csv.QUOTE_NONE, escapechar = ' ')


ft_model = fasttext.train_supervised('train.txt', lr=1.0, epoch=25, wordNgrams=2)


def generate_sentence_vectors(text):
    return ft_model.get_sentence_vector(text)


train_vectors = np.array(train_df['text'].apply(generate_sentence_vectors).tolist())
test_vectors = np.array(test_df['text'].apply(generate_sentence_vectors).tolist())


train_features = np.concatenate([train_vectors, train_df[['avg_word_length', 'avg_sent_length', 'vocab_richness', 'repetition_rate']].values], axis=1)
test_features = np.concatenate([test_vectors, test_df[['avg_word_length', 'avg_sent_length', 'vocab_richness', 'repetition_rate']].values], axis=1)


scaler = StandardScaler()
train_features = scaler.fit_transform(train_features)
test_features = scaler.transform(test_features)


lr_model = LogisticRegression(random_state=RANDOM_SEED)
lr_model.fit(train_features, train_df['label'])


test_preds = lr_model.predict(test_features)



true_labels = test_df['label']
predicted_labels = test_preds

# Calculate metrics
accuracy = accuracy_score(true_labels, predicted_labels)
f1 = f1_score(true_labels, predicted_labels, average='weighted')  
precision = precision_score(true_labels, predicted_labels, average='weighted')  
recall = recall_score(true_labels, predicted_labels, average='weighted')

# Print metrics
print('Accuracy:', accuracy)
print('F1 Score:', f1)
print('Precision:', precision)
print('Recall:', recall)


Read 1M words
Number of words:  82462
Number of labels: 0
Progress: 100.0% words/sec/thread: 1145329 lr:  0.000000 avg.loss:      -nan ETA:   0h 0m 0s


Accuracy: 0.6576070901033974
F1 Score: 0.6567209727944139
Precision: 0.663134158503344
Recall: 0.6576070901033974


In [7]:
le = LabelEncoder()
y_train_encoded = le.fit_transform(train_df['label'])
y_test_encoded = le.transform(test_df['label'])


dtrain = xgb.DMatrix(train_features, label=y_train_encoded)
dtest = xgb.DMatrix(test_features, label=y_test_encoded)


parameters = {
    'objective': 'multi:softmax',
    'num_class': 6,  
    'max_depth': 5,
    'eta': 0.3,
    'seed': RANDOM_SEED,  
    'silent': 1,  
}


xgb_model = xgb.train(parameters, dtrain, num_boost_round=10)


y_pred = xgb_model.predict(dtest)


accuracy = accuracy_score(y_test_encoded, y_pred)
precision = precision_score(y_test_encoded, y_pred, average='macro')
recall = recall_score(y_test_encoded, y_pred, average='macro')
f1 = f1_score(y_test_encoded, y_pred, average='macro')

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

<frozen importlib._bootstrap>:219: RuntimeWarning: pyarrow.lib.IpcWriteOptions size changed, may indicate binary incompatibility. Expected 72 from C header, got 88 from PyObject


[07:03:01] WARNING: ../src/learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[07:03:02] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softmax' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Accuracy: 0.7140324963072379
Precision: 0.7152465654162138
Recall: 0.7151946377337566
F1 Score: 0.7140318723708784
